<a href="https://colab.research.google.com/github/jhlee508/Colab/blob/master/2021_ai_online_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 런타임 방지
function ClickConnect(){
    console.log("코랩 연결 끊김 방지"); 
    document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect, 60 * 1000)

# 라이브러리

In [95]:
from glob import glob
import os
import cv2
import json

# Train 이미지 개수

In [93]:
# train 이미지 그룹 선택
train_number = 2

In [96]:
img_list = glob('/content/drive/Shareddrives/ay/train_{}/images/*.jpg'.format(train_number))
print("train_{} 이미지 개수:".format(train_number), len(img_list))

train_2 이미지 개수: 0


# Class 역순

In [ ]:
reverse_names = { 'eye_opened':0 , 'eye_closed':1, 'mouth_opened':2,'mouth_closed' : 3, 'face' : 4, 'phone':5, 'cigar': 6}
reverse_names

{'cigar': 6,
 'eye_closed': 1,
 'eye_opened': 0,
 'face': 4,
 'mouth_closed': 3,
 'mouth_opened': 2,
 'phone': 5}

In [ ]:
names = { 0:'eye_opened', 1:'eye_closed', 2:'mouth_opened', 3:'mouth_closed', 4:'face', 5:'phone', 6:'cigar' }
names

{0: 'eye_opened',
 1: 'eye_closed',
 2: 'mouth_opened',
 3: 'mouth_closed',
 4: 'face',
 5: 'phone',
 6: 'cigar'}

# Train Val Split

In [ ]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=10)

print(len(train_img_list), len(val_img_list))

19176 4795


In [ ]:
with open('/content/drive/Shareddrives/ay/train_{}/train.txt'.format(train_number), 'w') as f:
    f.write('\n'.join(train_img_list) + '\n')

with open('/content/drive/Shareddrives/ay/train_{}/val.txt'.format(train_number), 'w') as f:
    f.write('\n'.join(val_img_list) + '\n')

In [ ]:
import yaml

with open('/content/drive/Shareddrives/ay/task03_baseline/data/drowsy_face.yaml', 'r') as f:
    data = yaml.load(f)

print(data)

data['train'] = '/content/drive/Shareddrives/ay/train_{}/train.txt'.format(train_number)
data['val'] = '/content/drive/Shareddrives/ay/train_{}/val.txt'.format(train_number)

with open('/content/drive/Shareddrives/ay/train_{}/drowsy_face.yaml'.format(train_number), 'w') as f:
    yaml.dump(data, f)

print(data)

{'names': ['eye_opened', 'eye_closed', 'mouth_opened', 'mouth_closed', 'face', 'phone', 'cigar'], 'nc': 7, 'train': '/content/drive/Shareddrives/ay/train.txt', 'val': '/content/drive/Shareddrives/ay/val.txt'}
{'names': ['eye_opened', 'eye_closed', 'mouth_opened', 'mouth_closed', 'face', 'phone', 'cigar'], 'nc': 7, 'train': '/content/drive/Shareddrives/ay/train_1/train.txt', 'val': '/content/drive/Shareddrives/ay/train_1/val.txt'}


# YOLOv5 Clone

In [ ]:
%cd /content/drive/Shareddrives/ay/

!git clone https://github.com/ultralytics/yolov5.git

# YOLOv5 환경 설정

In [5]:
%cd /content/drive/Shareddrives/ay/yolov5

/content/drive/Shareddrives/ay/yolov5


In [6]:
!pip install -r requirements.txt

     |████████████████████████████████| 645kB 4.1MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# YOLOv5 Training

In [ ]:
%cd /content/drive/Shareddrives/ay/yolov5

!python train.py --batch 16 --epochs 20 --data /content/drive/Shareddrives/ay/train_1/drowsy_face.yaml --cfg /content/drive/Shareddrives/ay/yolov5/models/yolov5x.yaml --weights yolov5x.pt --name train_1_batch16_epoch20_yolov5x

/content/drive/Shareddrives/ay/yolov5
train: weights=yolov5x.pt, cfg=/content/drive/Shareddrives/ay/yolov5/models/yolov5x.yaml, data=/content/drive/Shareddrives/ay/train_1/drowsy_face.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=20, batch_size=16, img_size=[640, 640], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket=, cache_images=False, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=train_1_batch16_epoch20_yolov5x, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1
github: Command 'git fetch && git config --get remote.origin.url' timed out after 5 seconds, for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 v5.0-228-gae4261c torch 1.9.0+cu102 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

hyperparameters: lr0=0.01, lrf=0.2, momentu

In [ ]:
from google.colab import files

files.download('/content/drive/Shareddrives/ay/yolov5/runs/train/train_1_batch8_epoch10_yolov5s/weights/best.pt')

# Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/Shareddrives/ay/yolov5/runs/train/train_1_batch8_epoch10_yolov5s

# Detect

In [11]:
%cd /content/drive/Shareddrives/ay/yolov5

!python detect.py --weights /content/drive/Shareddrives/ay/yolov5/runs/train/train_1_batch16_epoch20_yolov5x/weights/best.pt --conf 0.3 --source "/content/drive/Shareddrives/ay/for_testing_30/images/000001.jpg"

/content/drive/Shareddrives/ay/yolov5
detect: weights=['/content/drive/Shareddrives/ay/yolov5/runs/train/train_1_batch16_epoch20_yolov5x/weights/best.pt'], source=/content/drive/Shareddrives/ay/for_testing_30/images/000001.jpg, imgsz=640, conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 v5.0-228-gae4261c torch 1.9.0+cu102 CUDA:0 (Tesla V100-SXM2-16GB, 16160.5MB)

Fusing layers... 
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(

In [2]:
!rm -r /content/drive/Shareddrives/ay/yolov5/train_1_batch16_epoch20_yolov5x_test

# Test

In [ ]:
# test.yaml 파일 생성
x = box[0]
y = box[1]
w = box[2]
h = box[3]
box = [x, y, w, h]
box[0] = [x-w/2, y-h/2, x+w/2,y+h/2]

In [10]:
%cd /content/drive/Shareddrives/ay/yolov5

!python test_custom.py --data /content/drive/Shareddrives/ay/for_testing_30/test.yaml --weights /content/drive/Shareddrives/ay/yolov5/runs/train/train_1_batch16_epoch20_yolov5x/weights/best.pt --batch-size 16 --task test --save-json --project train_for30_batch16_epoch20_yolov5x_test

/content/drive/Shareddrives/ay/yolov5
test: data=/content/drive/Shareddrives/ay/for_testing_30/test.yaml, weights=['/content/drive/Shareddrives/ay/yolov5/runs/train/train_1_batch16_epoch20_yolov5x/weights/best.pt'], batch_size=16, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=train_for30_batch16_epoch20_yolov5x_test, name=exp, exist_ok=False, half=False
YOLOv5 🚀 v5.0-228-gae4261c torch 1.9.0+cu102 CUDA:0 (Tesla V100-SXM2-16GB, 16160.5MB)

Fusing layers... 
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil

# Sample Submission 생성

In [82]:
import json
from tqdm import tqdm

In [90]:
with open('/content/drive/Shareddrives/ay/yolov5/train_for30_batch16_epoch20_yolov5x_test/exp5/best_predictions.json', 'r') as f:
    jdict = json.load(f)

with open('/content/drive/Shareddrives/ay/task03_baseline/test/sample_submission.json', 'r') as f:
    submission = json.load(f)

In [91]:
submission = submission['annotations']
jdict = jdict['annotations']

In [92]:
len(jdict)

35629

In [86]:
len(submission)

35631

In [87]:
for i in tqdm(range(len(submission))):
    for j in range(len(jdict)):
        if submission[i]['file_name'] == jdict[j]['file_name']:
            submission[i]['objects'] = jdict[j]['objects']

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.




 81%|████████  | 28822/35631 [07:17<01:52, 60.73it/s]




 81%|████████  | 28829/35631 [07:17<01:52, 60.44it/s]




 81%|████████  | 28836/35631 [07:17<01:55, 58.88it/s]




 81%|████████  | 28842/35631 [07:17<01:55, 58.59it/s]




 81%|████████  | 28848/35631 [07:17<01:57, 57.59it/s]




 81%|████████  | 28854/35631 [07:17<01:56, 58.03it/s]




 81%|████████  | 28860/35631 [07:17<01:59, 56.89it/s]




 81%|████████  | 28866/35631 [07:17<01:58, 57.03it/s]




 81%|████████  | 28872/35631 [07:17<01:58, 57.12it/s]




 81%|████████  | 28878/35631 [07:18<01:58, 56.94it/s]




 81%|████████  | 28885/35631 [07:18<01:54, 58.72it/s]




 81%|████████  | 28891/35631 [07:18<01:54, 58.86it/s]




 81%|████████  | 28898/35631 [07:18<01:52, 60.10it/s]




 81%|████████  | 28905/35631 [07:18<01:53, 59.51it/s]




 81%|████████  | 28911/35631 [07:18<01:54, 58.83it/s]




 81%|████████  | 28917/35631 [07:18<01:54, 58.67it/s]




 81%|████████  | 28924/35631 [07

In [89]:
print(submission[0]['objects'])

[{'class': 'face', 'position': [-92.054, 262.307, 273.601, 678.377], 'confidence_score': 0.95}, {'class': 'eye_opened', 'position': [191.795, 513.011, 276.442, 553.286], 'confidence_score': 0.89}, {'class': 'mouth_closed', 'position': [83.908, 702.525, 206.564, 749.379], 'confidence_score': 0.82}, {'class': 'eye_opened', 'position': [75.985, 525.069, 148.787, 562.428], 'confidence_score': 0.82}]


# Submission